In [1]:
from pydantic import BaseModel

In [2]:
class Point(BaseModel):
    title: str
    stayTime: int
    toleranceTime: int = 1
    
    class Config:
        frozen = True

In [3]:
class Section(BaseModel):
    pointFrom: Point
    pointTo: Point
    dateFrom: int
    dateTo: int
    cost: float
    
    class Config:
        frozen = True

In [4]:
points = {
    'A': Point(
        title = 'A',
        stayTime = 1,
    ),
    'B': Point(
        title = 'B',
        stayTime = 3,
    ),
}

In [5]:
sections = {
    Section(
        pointFrom = points['A'],
        pointTo = points['B'],
        dateFrom = 1,
        dateTo = 2,
        cost = 100.,
    ), # type: ignore
    Section(
        pointFrom = points['B'],
        pointTo = points['A'],
        dateFrom = 5,
        dateTo = 7,
        cost = 100.,
    ), # type: ignore
}

In [6]:
departures = {p: {s for s in sections if s.pointFrom is p} for p in points.values()} # type: ignore

In [11]:
def find_edges(
    currentPath: list[Section],
    departures: dict[Point, set[Section]],
) -> set[Section]:
    assert currentPath
    lastSection = currentPath[-1]
    
    dateStart = lastSection.dateTo + lastSection.pointTo.stayTime - lastSection.pointTo.toleranceTime
    dateEnd = lastSection.dateTo + lastSection.pointTo.stayTime + lastSection.pointTo.toleranceTime
    
    visited = {s.pointTo for s in currentPath[1:]} # type: ignore
    
    return {s for s in sections if dateStart <= s.dateFrom <= dateEnd and s.pointTo not in visited} # type: ignore

In [12]:
def find_paths(
    currentPath: list[Section],
    departures: dict[Point, set[Section]],
) -> list[list[Section]]:
    
    if edges := find_edges(
        currentPath = currentPath,
        departures = departures,
    ):
        allPaths: list[list[Section]] = []
        for e in edges:
            allPaths += find_paths(
                currentPath = currentPath + [e],
                departures = departures,
            )
        return allPaths
    else:
        return [currentPath]

In [13]:
def build_init_path(
    pointInit: Point,
    dateInit: int,
) -> list[Section]:
    return [Section(
        pointFrom = pointInit,
        pointTo = pointInit,
        dateFrom = dateInit,
        dateTo = dateInit,
        cost = 0.,
    )]

In [14]:
find_paths(build_init_path(points['A'], 0), departures)

[[Section(pointFrom=Point(title='A', stayTime=1, toleranceTime=1), pointTo=Point(title='A', stayTime=1, toleranceTime=1), dateFrom=0, dateTo=0, cost=0.0),
  Section(pointFrom=Point(title='A', stayTime=1, toleranceTime=1), pointTo=Point(title='B', stayTime=3, toleranceTime=1), dateFrom=1, dateTo=2, cost=100.0),
  Section(pointFrom=Point(title='B', stayTime=3, toleranceTime=1), pointTo=Point(title='A', stayTime=1, toleranceTime=1), dateFrom=5, dateTo=7, cost=100.0)]]